In [2]:
import numpy as np

In [57]:
def pad_input_matrix(X, pad: int, padding_value: int = 0):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image

    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C) # m is omitted for now
    """
    X_pad = np.pad(X, pad_width=((0, 0), (pad, pad), (pad, pad)),
                   mode='constant', constant_values=(padding_value, padding_value))
    # X_pad = np.pad(X, ((0,0),(pad,pad),(pad,pad),(0,0)))

    return X_pad

In [ ]:
class ConvolutionalLayer:
    # number of output channel = filter counts
    def __init__(self, input_shape: (int, int, int), number_of_output_channel: int, filter_dimension: int,
                 stride: int, padding: int):
        self.input = None
        self.output = None
        self.number_of_output_channel = number_of_output_channel
        self.filter_dimension = filter_dimension
        self.stride = stride
        self.padding = padding

        input_depth, input_height, input_width = input_shape
        self.input_shape = input_shape
        self.input_depth = input_depth

        self.output_shape = (number_of_output_channel, input_height - filter_dimension + 1, input_width
                             - filter_dimension + 1)

        self.filters_shape = (number_of_output_channel, input_depth, filter_dimension, filter_dimension)
        self.filters = np.random.randn(*self.filters_shape)

        self.biases = np.random.randn(*self.output_shape)


    def convolution_single_step(input_slice, filter_mat):
        """
        Apply one filter defined by parameters weight on a single slice (a_slice_prev) of the output activation
        of the previous layer.

        Arguments:
        a_slice_prev -- slice of input data of shape (f, f, n_C_prev)
        weight -- Weight parameters contained in a window - matrix of shape (f, f, n_C_prev)
        bias -- Bias parameters contained in a window - matrix of shape (1, 1, 1)

        Returns:
        Z -- a scalar value, the result of convolving the sliding window (weight, bias) on a slice x of the input data
        """

        #(≈ 3 lines of code)
        # Element-wise product between a_slice_prev and weight. Do not add the bias yet.
        s = np.multiply(a_slice_prev, weight)
        # Sum over all entries of the volume s.
        Z = np.sum(s)
        # Add bias to Z. Cast bias to a float() so that Z results in a scalar value.
        bias = np.squeeze(bias)
        Z = Z + bias

        return Z


    def conv_forward(self, A_prev, W, b):
        """
        Implements the forward propagation for a convolution function

        Arguments:
        A_prev -- output activations of the previous layer,
            numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
        W -- Weights, numpy array of shape (f, f, n_C_prev, n_C)
        b -- Biases, numpy array of shape (1, 1, 1, n_C)
        hparameters -- python dictionary containing "stride" and "pad"

        Returns:
        Z -- conv output, numpy array of shape (m, n_H, n_W, n_C)
        cache -- cache of values needed for the conv_backward() function
        """

        # Retrieve dimensions from A_prev's shape (≈1 line)
        (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape

        # Retrieve dimensions from W's shape (≈1 line)
        (f, f, n_C_prev, n_C) = W.shape

        # Retrieve information from "hparameters" (≈2 lines)
        # stride = hparameters["stride"]
        # pad = hparameters["pad"]

        # Compute the dimensions of the CONV output volume using the formula given above.
        # Hint: use int() to apply the 'floor' operation. (≈2 lines)
        n_H = int((n_H_prev + 2*self.padding - f)/self.stride) + 1
        n_W = int((n_W_prev + 2*self.padding - f)/self.stride) + 1

        # Initialize the output volume Z with zeros. (≈1 line)
        Z = np.zeros((m, n_H, n_W, n_C))

        # Create A_prev_pad by padding A_prev
        A_prev_pad = zero_pad(A_prev, self.padding)

        for i in range(m):               # loop over the batch of training examples
            a_prev_pad = A_prev_pad[i]          # Select ith training example's padded activation
            for h in range(n_H):           # loop over vertical axis of the output volume
                # Find the vertical start and end of the current "slice" (≈2 lines)
                vert_start = self.stride * h
                vert_end = vert_start  + f

                for w in range(n_W):       # loop over horizontal axis of the output volume
                    # Find the horizontal start and end of the current "slice" (≈2 lines)
                    horiz_start = self.stride * w
                    horiz_end = horiz_start + f

                    for c in range(n_C):   # loop over channels (= #filters) of the output volume

                        # Use the corners to define the (3D) slice of a_prev_pad (See Hint above the cell). (≈1 line)
                        a_slice_prev = a_prev_pad[vert_start:vert_end,horiz_start:horiz_end,:]

                        # Convolve the (3D) slice with the correct filter W and bias b, to get back one output neuron. (≈3 line)
                        weights = W[:, :, :, c]
                        biases  = b[:, :, :, c]
                        Z[i, h, w, c] = self.convolution_single_step(a_slice_prev, weights, biases)

        # Save information in "cache" for the backprop
        cache = (A_prev, W, b, hparameters)

        return Z, cache
    


In [41]:
np.random.seed(1)
x = np.random.randn(2, 2)
s, _ = x.shape
print(x[0])
print(s)

[ 1.62434536 -0.61175641]
2


In [34]:
temp = np.floor(3.7)
print(temp)

3.0


In [7]:
a = [[1, 2], [3, 40]]
a = np.pad(a, pad_width=(1, 1)  mode='constant', constant_values=(0, 0))
print(a)

[[ 0  0  0  0]
 [ 0  1  2  0]
 [ 0  3 40  0]
 [ 0  0  0  0]]


In [100]:
def flatten_matrix(input):
    return input.flatten().reshape(-1, 1)

x = np.random.randn(3, 2, 2)
y = flatten_matrix(x)
print(x)
print(y)
print(y.shape)

[[[-0.91797604 -0.85183742]
  [ 0.09075857  1.16220247]]

 [[ 0.32178247  1.34131783]
  [ 0.14167518  1.73738467]]

 [[ 0.37576878  1.47825568]
  [-2.08016719 -0.45342757]]]
[[-0.91797604]
 [-0.85183742]
 [ 0.09075857]
 [ 1.16220247]
 [ 0.32178247]
 [ 1.34131783]
 [ 0.14167518]
 [ 1.73738467]
 [ 0.37576878]
 [ 1.47825568]
 [-2.08016719]
 [-0.45342757]]
(12, 1)


In [87]:

x = np.random.randint(low = 1, high = 8, size=(2, 2))
y = np.random.randint(low = 1, high = 8, size=(2, 2))
print(x)
print(y)
print(np.multiply(x, y))
print(np.sum(np.multiply(x, y)))

[[2 4]
 [5 1]]
[[2 4]
 [5 3]]
[[ 4 16]
 [25  3]]
48


In [101]:
kernel = np.random.randint(low=1, high=10, size=(3,3))
print(kernel)
kernel = np.flipud(np.fliplr(kernel))
print(kernel)

[[8 9 9]
 [9 1 8]
 [3 1 8]]
[[8 1 3]
 [8 1 9]
 [9 9 8]]


In [103]:
import numpy as np


def file_read(file_path: str):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    lines = [line.split() for line in lines]
    lines = [[int(x) if idx != 0 else x for idx, x in enumerate(line)] for line in lines]

    '''
    *********************************** INPUT FORMAT *********************************************
    Conv: the number of output channels, filter dimension, stride, padding.
    ReLU: Activation layer.
    Pool: filter dimension, stride.
    FC: output dimension.
    Flattening layer: it will convert a (series of) convolutional filter maps to a column vector.
    Softmax: it will convert final layer projections to normalized probabilities.
    '''

    return lines


def pad_input_2d_mat(X, pad: int, padding_value: int = 0):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image

    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions

    Returns:
    X_pad -- padded image of shape (m, n_H + 2 * pad, n_W + 2 * pad, n_C) # m is omitted for now
    """
    X_pad = np.pad(X, pad_width=((pad, pad), (pad, pad)),
                   mode='constant', constant_values=(padding_value, padding_value))
    # X_pad = np.pad(X, ((0,0),(pad,pad),(pad,pad),(0,0)))

    return X_pad


def cross_correlation2d(input_mat, filter_mat, stride, padding, result_shape):
    filter_dimension, _ = filter_mat.shape
    _, result_height, result_width = result_shape

    result = np.zeros((result_height, result_width), dtype=float)

    if padding != 0:
        pad_input = pad_input_2d_mat(input_mat, pad=padding)
    else:
        pad_input = input_mat

    for h in range(result_height):  # loop over vertical axis of the output volume
        # Find the vertical start and end of the current "slice" (≈2 lines)
        vertical_start = stride * h
        vertical_end = vertical_start + filter_dimension

        for w in range(result_width):  # loop over horizontal axis of the output volume
            # Find the horizontal start and end of the current "slice" (≈2 lines)
            horizontal_start = stride * w
            horizontal_end = horizontal_start + filter_dimension

            input_slice = pad_input[vertical_start:vertical_end, horizontal_start:horizontal_end]
            result[h, w] = np.sum(np.multiply(input_slice, filter_mat))
            # print(result[h, w])

    return result


def convolution2d(input_mat, filter_mat, stride, padding, result_shape):
    # Flip the filter
    rotated_filter = np.flipud(np.fliplr(filter_mat))

    return cross_correlation2d(input_mat, rotated_filter, stride, padding, result_shape)


def flatten_matrix(input_data):
    return input_data.flatten().reshape(-1, 1)

In [5]:
import numpy as np
np.random.seed(1)
x = np.random.randn(1, 4, 4)
print(x)
x = x.flatten().reshape(-1, 1)
x = np.maximum(0, x)

print(x)

[[[ 1.62434536 -0.61175641 -0.52817175 -1.07296862]
  [ 0.86540763 -2.3015387   1.74481176 -0.7612069 ]
  [ 0.3190391  -0.24937038  1.46210794 -2.06014071]
  [-0.3224172  -0.38405435  1.13376944 -1.09989127]]]
[[1.62434536]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.86540763]
 [0.        ]
 [1.74481176]
 [0.        ]
 [0.3190391 ]
 [0.        ]
 [1.46210794]
 [0.        ]
 [0.        ]
 [0.        ]
 [1.13376944]
 [0.        ]]


In [12]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
from keras.utils import np_utils
import numpy as np
def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype("float32") / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

In [13]:
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)


In [14]:
print(x_train.shape)
print(x_test.shape)
print(x_test[100])

(200, 1, 28, 28)
(200, 1, 28, 28)
[[[0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.         0.         0.
   0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.         0.
   0.         0.         0.  

In [18]:
f_size = (1,2,3,4)
print(*f_size)
ara = np.random.randn(*f_size)
ara1 = np.random.sample(f_size)

print(ara)
print(ara1)



1 2 3 4
[[[[ 0.20937219 -1.14722491 -1.73588297  0.06441288]
   [-0.89364831 -1.45206864 -0.05235459 -1.45838667]
   [ 1.09016717  2.43782301 -1.20939821  1.15258488]]

  [[ 0.51603118 -0.08547172 -0.77304463 -1.82140248]
   [-0.02846136 -0.22934414 -0.16255177 -2.11475618]
   [-0.91421322  0.25813814  0.6433904   0.75538222]]]]
[[[[0.63817856 0.69648054 0.11644818 0.776604  ]
   [0.40350945 0.49764749 0.53628667 0.70427408]
   [0.37755531 0.94384    0.45245379 0.99497421]]

  [[0.4470628  0.43569936 0.9552655  0.49652635]
   [0.44797672 0.33654175 0.03783999 0.05457376]
   [0.89767449 0.98379996 0.5414832  0.65960349]]]]
